In [3]:
# Variables explaination link: https://api.census.gov/data/2010/dec/sf1/variables.html

"""
P001001	Total	TOTAL POPULATION	not required	P001001ERR	0	(not a predicate)	P1
P002001	Total	URBAN AND RURAL	not required	P002001ERR	0	int	P2
P002002	Total!!Urban	URBAN AND RURAL	not required		0	int	P2
P002003	Total!!Urban!!Inside urbanized areas	URBAN AND RURAL	not required		0	int	P2
P002004	Total!!Urban!!Inside urban clusters	URBAN AND RURAL	not required		0	int	P2
P002005	Total!!Rural	URBAN AND RURAL	not required		0	int	P2
P002006	Total!!Not defined for this file	URBAN AND RURAL	not required		0	int	P2
P003001	Total	RACE	not required	P003001ERR	0	int	P3
P003002	Total!!White alone	RACE	not required		0	int	P3
P003003	Total!!Black or African American alone	RACE	not required		0	int	P3
P003004	Total!!American Indian and Alaska Native alone	RACE	not required		0	int	P3
P003005	Total!!Asian alone	RACE	not required		0	int	P3
P003006	Total!!Native Hawaiian and Other Pacific Islander alone	RACE	not required		0	int	P3
P003007	Total!!Some Other Race alone	RACE	not required		0	int	P3
P003008	Total!!Two or More Races	RACE	not required		0	int	P3
P004001	Total	HISPANIC OR LATINO ORIGIN	not required	P004001ERR	0	int	P4
P004002	Total!!Not Hispanic or Latino	HISPANIC OR LATINO ORIGIN	not required		0	int	P4
P004003	Total!!Hispanic or Latino	HISPANIC OR LATINO ORIGIN
"""

# Dataset: https://api.census.gov/data/2010/dec/sf1?get=P001001,P003002,P003003,P004003,NAME&for=county:*
import pandas as pd

In [4]:
df = pd.read_json("https://api.census.gov/data/2010/dec/sf1?get=P001001,P003002,P003003,P004003,NAME&for=county:*")


new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
new_header[0] = 'Total'
new_header[1] = 'White'
new_header[2] = 'Black'
new_header[3] = 'Hispanic or Latino'
df.columns = new_header #set the header row as the df header
df

,Total,White,Black,Hispanic or Latino,NAME,state,county
1,125744,96344,8019,15445,"Sebastian County, Arkansas",05,131
2,17058,11949,734,5220,"Sevier County, Arkansas",05,133
3,17264,16582,93,290,"Sharp County, Arkansas",05,135
4,12394,11995,11,157,"Stone County, Arkansas",05,137
5,41639,26276,13721,1460,"Union County, Arkansas",05,139
...,...,...,...,...,...,...,...
3217,98736,91946,874,1804,"Eau Claire County, Wisconsin",55,035
3218,4423,4306,10,37,"Florence County, Wisconsin",55,037
3219,101633,95674,1305,4368,"Fond du Lac County, Wisconsin",55,039
3220,9304,7690,76,138,"Forest County, Wisconsin",55,041


In [5]:
df['NAME']

1          Sebastian County, Arkansas
2             Sevier County, Arkansas
3              Sharp County, Arkansas
4              Stone County, Arkansas
5              Union County, Arkansas
                    ...              
3217     Eau Claire County, Wisconsin
3218       Florence County, Wisconsin
3219    Fond du Lac County, Wisconsin
3220         Forest County, Wisconsin
3221      Jefferson County, Wisconsin
Name: NAME, Length: 3221, dtype: object

In [6]:
df[False == df['NAME'].str.contains("County")]

,Total,White,Black,Hispanic or Latino,NAME,state,county
107,97581,75175,4423,5651,"Fairbanks North Star Borough, Alaska",02,090
111,3141,660,219,385,"Aleutians East Borough, Alaska",02,013
112,5561,2004,332,726,"Aleutians West Census Area, Alaska",02,016
113,1826,1637,10,42,"Denali Borough, Alaska",02,068
114,4847,878,11,101,"Dillingham Census Area, Alaska",02,070
...,...,...,...,...,...,...,...
2888,9301,5456,2617,8775,"Vieques Municipio, Puerto Rico",72,147
2889,26073,21405,2212,25983,"Villalba Municipio, Puerto Rico",72,149
2890,37941,24888,5334,37686,"Yabucoa Municipio, Puerto Rico",72,151
2891,42043,34899,2482,41831,"Yauco Municipio, Puerto Rico",72,153


After some research, we can observe that for instance, we have the datas for Carson City, from Nevada.
This City is not part of any county (source: https://en.wikipedia.org/wiki/Carson_City,_Nevada). 
// TODO réfléchir à quoi faire de ces données, et peut être justifier qu'on a que des County dans notre stop data set (si c le cas)

In [58]:
def getStatsByCounty(countyName, stateName, df):
    # Split the NAME column in "County/City Name" and "State"
    new_df = df.copy()
    new_df[['County_name', 'State_name']] = df['NAME'].str.split(',', 1, expand=True)
    matches = new_df[new_df['County_name'].str.contains(countyName)].copy()
    matches = matches[matches['State_name'].str.contains(stateName)].copy()
    
    matches['Total'] = pd.to_numeric(matches['Total'])
    matches['White'] = pd.to_numeric(matches['White'])
    matches['Black'] = pd.to_numeric(matches['Black'])
    matches['Hispanic or Latino'] = pd.to_numeric(matches['Hispanic or Latino'])
    
    if(len(matches) == 0):
        print("[getStatsByCounty] Error: County not found: ", countyName)
        return;
    if(len(matches) == 1):
        return {'total': matches['Total'].values[0], 
                'white': (matches['White']/matches['Total']).values[0],
                'black': (matches['Black']/matches['Total']).values[0], 
             #   'hispanicy or latino': (matches['Hispanic or Latino']/matches['Total']).values[0]
               }
    else:
        print("[getStatyByCount] Error: Too many matches: ", countyName)
        print(matches['County_name'])



In [59]:
getStatsByCounty('Miami', 'Florida', df)

{'total': 2496435, 'white': 0.7378069126574496, 'black': 0.18946057077392361}